# Пример использования PaperResearchAgent

Этот ноутбук демонстрирует работу с агентом для поиска и саммаризации научных статей.


## 1. Импорт необходимых библиотек


In [1]:
from mas_automl.agents import PaperResearchAgent
import arxiv
import asyncio


## 2. Инициализация агента

Создаём экземпляр `PaperResearchAgent`, который будет искать и саммаризировать научные статьи.


In [2]:
# Создаём агента в обычном режиме
agent = PaperResearchAgent()
test_agent = PaperResearchAgent(test_mode=True)


## 3. Поиск и саммаризация статей

Агент использует LangGraph для оркестрации процесса:
1. Поиск статей в arXiv
2. Саммаризация найденных статей с помощью LLM

### 3.1. Формирование запросов для поиска статей о фреймворках

Для поиска статей о фреймворках, бенчмарках и тестировании AutoML систем (например, как "AMLB: an AutoML Benchmark"), включите в запрос соответствующие ключевые слова:
- `framework`, `frameworks`
- `benchmark`, `benchmarking`
- `evaluation`, `evaluating`
- `tool`, `tools`
- `system`, `systems`
- `library`, `libraries`
- `platform`, `platforms`


## 3.3. Поиск новых статей (сортировка по дате)

Для поиска только новых статей можно использовать сортировку по дате подачи и фильтрацию по времени:

- `sort_by=arxiv.SortCriterion.SubmittedDate` - сортировка по дате подачи (новые сначала)
- `days_back=N` - фильтр: только статьи за последние N дней

Это особенно полезно для поиска свежих публикаций об AutoML фреймворках и бенчмарках.


### Доступные варианты сортировки:

- `arxiv.SortCriterion.Relevance` (по умолчанию) - сортировка по релевантности запросу
- `arxiv.SortCriterion.SubmittedDate` - сортировка по дате подачи статьи (новые сначала)
- `arxiv.SortCriterion.LastUpdatedDate` - сортировка по дате последнего обновления

### Примеры использования фильтра по дате:

- `days_back=30` - только статьи за последний месяц
- `days_back=90` - только статьи за последние 3 месяца  
- `days_back=365` - только статьи за последний год


In [9]:
# Пример: поиск новых статей об AutoML фреймворках за последний год
# Сортировка по дате подачи (новые сначала) и фильтр: только статьи за последние 365 дней
query_new = "AutoML framework benchmark library evaluation"
result_new = await test_agent.handle(
    query_new,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    days_back=365 * 2  # Только статьи за последний год
)

print(f"Запрос: {result_new.get('query', '')}")
print(f"Найдено статей: {len(result_new.get('summaries', []))}")
print("\nНовые статьи о фреймворках и бенчмарках (отсортированы по дате):")
for i, summary in enumerate(result_new.get('summaries', []), 1):
    print(f"\n{i}. {summary.get('title', 'N/A')}")
    print(f"   Дата публикации: {summary.get('published_date', 'N/A')}")
    print(f"   URL: {summary.get('url', 'N/A')}")


Запрос: AutoML framework benchmark library evaluation
Найдено статей: 2

Новые статьи о фреймворках и бенчмарках (отсортированы по дате):

1. AutoML-Agent: A Multi-Agent LLM Framework for Full-Pipeline AutoML
   Дата публикации: 2024-10-03 20:01:09+00:00
   URL: http://arxiv.org/abs/2410.02958v2

2. AutoGluon-Multimodal (AutoMM): Supercharging Multimodal AutoML with Foundation Models
   Дата публикации: 2024-04-24 22:28:12+00:00
   URL: http://arxiv.org/abs/2404.16233v2
